# Example Code for Estimating the Ground State Energy of Hydroxyl (·OH)

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [1]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 10.1 MB/s eta 0:00:00


In [2]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.8 MB/s eta 0:00:00


In [23]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer,pulse, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as seeds that requires to run, and the result will be calculated as the average of results from each seed. And please use shots as 4000.

In [2]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
iterations = 125
shot = 4000

## Generate Hamiltonian and Pauli String

At this step, the example code uses PySCF to generate the hamiltonian of hydroxyl with basis function as 'sto3g' to fit the spin orbital, then uses JordanWignerMapper to map the fermionic terms to pauli strings. To be noticed, other chemistry tool also allowed to be used at this step, but keep in mind to use 'sto-3g' and Jordan Wigner Mapper which should gives 12 qubits and 631 paulil terms.

In [3]:
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()

/usr/local/lib/python3.10/dist-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [4]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)
second_q_op = hamiltonian.second_q_op()

Polynomial Tensor
 "+-":
[[-3.21461222e+01  5.59899100e-01  1.87617178e-01  3.33659548e-16
   8.16555038e-16 -1.94702445e-01]
 [ 5.59899100e-01 -7.35898345e+00 -2.46352634e-01 -8.92115048e-16
  -2.69147474e-15  9.51226718e-01]
 [ 1.87617178e-01 -2.46352634e-01 -6.56995119e+00  3.21647931e-15
   4.70241444e-15 -1.09726793e+00]
 [ 3.33752208e-16 -9.17086521e-16  3.35069276e-15 -6.94886145e+00
  -9.86529571e-16 -1.45773318e-15]
 [ 8.19334801e-16 -2.63222381e-15  4.31569054e-15 -1.05070629e-15
  -6.94886145e+00 -7.68897862e-15]
 [-1.94702445e-01  9.51226718e-01 -1.09726793e+00 -1.62239641e-15
  -7.50250929e-15 -4.64967973e+00]]
 "++--":
[[[[ 4.74977044e+00 -4.38465691e-01 -1.51436760e-01 -2.70572717e-16
    -6.49186366e-16  1.59790984e-01]
   [-4.38465691e-01  6.47204045e-02  1.84429506e-02  3.49797616e-17
     1.01610685e-16 -2.66865302e-02]
   [-1.51436760e-01  1.84429506e-02  2.46189939e-02 -1.18769659e-17
    -2.34635394e-17  6.40512562e-03]
   [-2.70689779e-16  3.49109142e-17 -1.17322

In [5]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op)

<ipython-input-5-60b4e97714fb>:2: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
/usr/local/lib/python3.10/dist-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


We recommend to use classical minimum eigensolver to obtain a reference energy at this step. In case some of the classical minimum eigensolver donot directly gives nuclear repulsion energy, we give reference energies below: *Comupted Energy*: -78.75252123, *Nuclear Repulsion_energy*: 4.36537496654537. *Obtained Reference Ground State Energy*: -74.38714627.

In [6]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [7]:
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [8]:
print(result.nuclear_repulsion_energy)

4.36537496654537


In [9]:
ref_value = result.computed_energies + result.nuclear_repulsion_energy
print(ref_value)

[-74.38714627]


## Construct Ansatz

At this stage, you can implement various techniques to search good ansatz architecture which is important for variational quantum algorihms. Moreover, how to obtain a good initial state is a good topic to do research, we require participant to self-reflection there techniques (include the techniques for preprocessing ansatz or initial states) with maximum 10 points, and submit a short description for used techniques, we will have three graders to evaluate the techniques.

In [10]:
ansatz = UCCSD(
    qmolecule.num_spatial_orbitals,
    qmolecule.num_particles,
    mapper,
    initial_state=HartreeFock(
        qmolecule.num_spatial_orbitals,
        qmolecule.num_particles,
        mapper,
    ),
)


In [11]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [20]:
estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': 'GPU'
        # 'noise_model': noise_model
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler':seed_transpiler
    }
)

In [21]:
vqe_solver = VQE(estimator, ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [39]:
calc = GroundStateEigensolver(mapper, vqe_solver)
res = calc.solve(qmolecule)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -78.318374453615
  - computed part:      -78.318374453615
~ Nuclear repulsion energy (Hartree): 4.365374966545
> Total ground state energy (Hartree): -73.952999487069
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 8.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.85037676  -0.28818323  -1.59757447]
 
  0: 
  * Electronic dipole moment (a.u.): [0.39515867568  -0.133932189247  -0.742341180385]
    - computed part:      [0.39515867568  -0.133932189247  -0.742341180385]
  > Dipole moment (a.u.): [0.45521808432  -0.154251040753  -0.855233289615]  Total: 0.981040706358
                 (debye): [1.157048850129  -0.392067001471  -2.173785990552]  Total: 2.493556517897
 


## Calculate the Accuracy (Most Important Metric)

In [39]:
accuracy = res / ref_value
print(accuracy)

[0.99416369]


## Obtain the Duration of Quantum Circuit

In [26]:
from qiskit.providers.fake_provider import *
backend = FakeMontreal()

In [28]:
with pulse.build(backend) as my_program1:
  pulse.call(ansatz)

In [36]:
my_program1.duration

19859872